In [10]:
# For tips on running notebooks in Google Colab, see
# https://pytorch.org/tutorials/beginner/colab
%matplotlib inline

https://towardsdatascience.com/building-a-python-code-generator-4b476eec5804
https://github.com/divyam96/English-to-Python-Converter/blob/main/English_to_Python.ipynb
https://towardsdatascience.com/custom-datasets-in-pytorch-part-2-text-machine-translation-71c41a3e994e


In [11]:
import math
import os
from tempfile import TemporaryDirectory
from typing import Tuple
import pandas as pd
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
from torch import Tensor
from torch.utils.data import dataset, DataLoader
from torch.utils.tensorboard import SummaryWriter
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = "cpu"
print(device)



cpu


In [50]:
f = open("english_python_data.txt", "r")
file_lines = f.readlines()
#Build Dataset
dps = []
dp = None
questions = []
solutions = []
for line in file_lines:
  if line[0] == "#":
    if dp:
      dp['solution'] = ''.join(dp['solution'])
      dps.append(dp)
      solutions.append(dp['solution'])
    dp = {"question": None, "solution": []}
    dp['question'] = line[1:]
    questions.append(line[1:])
  else:
    dp["solution"].append(line)
  
print([[x,y] for x,y in zip(questions,solutions)][0])


[' write a python program to add two numbers \n', "num1 = 1.5\nnum2 = 6.3\nsum = num1 + num2\nprint(f'Sum: {sum}')\n\n\n"]


In [51]:
from tokenize import tokenize, untokenize
import io
def tokenize_python_code(python_code_str):
    python_tokens = list(tokenize(io.BytesIO(python_code_str.encode('utf-8')).readline))
    tokenized_output = []
    for i in range(0, len(python_tokens)):
        tokenized_output.append((python_tokens[i].type, python_tokens[i].string))
    return tokenized_output

#print(dps[0])
error_count = 0
errors = []
for i in solutions:
    try:
        tokenize_python_code(i)
    except Exception as e:
        errors.append((e,i))
        error_count += 1

print(error_count)
for e,i in errors:
    print(e)
    print(i)



0


In [52]:
python_problems_df = pd.DataFrame(dps)
python_problems_df.head()


,question,solution
0,write a python program to add two numbers \n,num1 = 1.5\nnum2 = 6.3\nsum = num1 + num2\npri...
1,write a python function to add two user provi...,"def add_two_numbers(num1, num2):\n sum = nu..."
2,write a program to find and print the largest...,\nnum1 = 10\nnum2 = 12\nnum3 = 14\nif (num1 >=...
3,write a program to find and print the smalles...,num1 = 10\nnum2 = 12\nnum3 = 14\nif (num1 <= n...
4,Write a python function to merge two given li...,"def merge_lists(l1, l2):\n return l1 + l2\n..."


In [53]:
python_problems_df.loc[9]

question     Write a program to print the multiplication t...
solution    \nnum = 9\nfor i in range(1, 11):\n   print(f"...
Name: 9, dtype: object

In [54]:
train_df = []
val_df = []
np.random.seed(0)
msk = np.random.rand(len(dps)) < .85 # Splitting data into 85% train and 15% validation
for truth,val in zip(msk,dps):
    if truth:
        train_df.append(val)
    else:
        val_df.append(val)

train_df = pd.DataFrame(train_df)
val_df = pd.DataFrame(val_df)



In [55]:
train_df[5:10]

,question,solution
5,Write a program to check whether a number is ...,"num = 337\n\nif num > 1:\n for i in range(2,..."
6,Write a python function that prints the facto...,"def print_factors(x):\n print(f""The factors ..."
7,Write a program to print the multiplication t...,"\nnum = 9\nfor i in range(1, 11):\n print(f""..."
8,"Write a python function to print powers of 2,...",def two_power(terms):\n result = list(map(l...
9,Write a program to filter the numbers in a li...,"my_list = [11, 45, 74, 89, 132, 239, 721, 21]\..."


In [142]:
class Vocabulary:
  
    def __init__(self, freq_threshold, max_size):
        '''
        freq_threshold : the minimum times a word must occur in corpus to be treated in vocab
        max_size : max source vocab size. Eg. if set to 10,000, we pick the top 10,000 most frequent words and discard others
        '''
        #initiate the index to token dict
        ## <PAD> -> padding, used for padding the shorter sentences in a batch to match the length of longest sentence in the batch
        ## <SOS> -> start token, added in front of each sentence to signify the start of sentence
        ## <EOS> -> End of sentence token, added to the end of each sentence to signify the end of sentence
        ## <UNK> -> words which are not found in the vocab are replace by this token
        self.itos = {0: '<PAD>', 1:'<SOS>', 2:'<EOS>', 3: '<UNK>'}
        #initiate the token to index dict
        self.stoi = {k:j for j,k in self.itos.items()} 
        
        self.freq_threshold = freq_threshold
        self.max_size = max_size
    
    '''
    __len__ is used by dataloader later to create batches
    '''
    def __len__(self):
        return len(self.itos)
    
    '''
    a simple tokenizer to split on space and converts the sentence to list of words
    '''
    @staticmethod
    def tokenizer(text):
        return [tok.lower().strip() for tok in text.split(' ')]
    
    '''
    build the vocab: create a dictionary mapping of index to string (itos) and string to index (stoi)
    output ex. for stoi -> {'the':5, 'a':6, 'an':7}
    '''
    def build_vocabulary(self, sentence_list):
        pass
            
    '''
    convert the list of words to a list of corresponding indexes
    '''    
    def numericalize_base(self, tokenized_text):
        numericalized_text = []
        for token in tokenized_text:
            if token in self.stoi.keys():
                numericalized_text.append(self.stoi[token])
            else: #out-of-vocab words are represented by UNK token index
                numericalized_text.append(self.stoi['<UNK>'])
                
        return numericalized_text

class Vocabulary_Output(Vocabulary):
    def build_vocabulary(self, sentence_list):
        

        frequencies = {}  #init the freq dict
        idx = 4 #index from which we want our dict to start. We already used 4 indexes for pad, start, end, unk
        
        for sentence in sentence_list:
            python_tokens = list(tokenize(io.BytesIO(sentence.encode('utf-8')).readline))
            tokenized_output = []
            #print([i.string for i in list(tokenize(io.BytesIO(sentence.encode('utf-8')).readline))])
            for i in range(0, len(python_tokens)):
                word = python_tokens[i].string
                # print(word)
                if word not in frequencies.keys():
                    frequencies[word]=1
                else:
                    frequencies[word]+=1

        #limit vocab by removing low freq words
        frequencies = {k:v for k,v in frequencies.items() if v>self.freq_threshold} 
        
        print(self.freq_threshold)

        #limit vocab to the max_size specified
        frequencies = dict(sorted(frequencies.items(), key = lambda x: -x[1])[:self.max_size-idx]) # idx =4 for pad, start, end , unk
        
        #create vocab
        for word in frequencies.keys():
            self.stoi[word] = idx
            self.itos[idx] = word
            idx+=1
    
    def numericalize(self, text):
        tokenized_text = [i.string for i in list(tokenize(io.BytesIO(text.encode('utf-8')).readline))]
        return self.numericalize_base(tokenized_text)
                
class Vocabulary_Input(Vocabulary):
    def build_vocabulary(self, sentence_list):
        #calculate the frequencies of each word first to remove the words with freq < freq_threshold
        frequencies = {}  #init the freq dict
        idx = 4 #index from which we want our dict to start. We already used 4 indexes for pad, start, end, unk
        
        #calculate freq of words
        for sentence in sentence_list:
            for word in self.tokenizer(sentence):
                if word not in frequencies.keys():
                    frequencies[word]=1
                else:
                    frequencies[word]+=1
                    
                    
        #limit vocab by removing low freq words
        frequencies = {k:v for k,v in frequencies.items() if v>self.freq_threshold} 
        
        #limit vocab to the max_size specified
        frequencies = dict(sorted(frequencies.items(), key = lambda x: -x[1])[:self.max_size-idx]) # idx =4 for pad, start, end , unk
        
        

        #create vocab
        for word in frequencies.keys():
            self.stoi[word] = idx
            self.itos[idx] = word
            idx+=1
    def numericalize(self, text):
        tokenized_text = self.tokenizer(text)
        return self.numericalize_base(tokenized_text)

In [143]:
#Test
voc = Vocabulary_Input(0,100000)
voc.build_vocabulary(questions)
#print(voc.stoi)
#print(voc.numericalize("write a python program to add two numbers"))

voc_out = Vocabulary_Output(0,10000)
voc_out.build_vocabulary(solutions)
#print(voc_out.itos)

print(len(voc_out.itos))



print(questions[1])
print(solutions[1])



0
3381
 write a python function to add two user provided numbers and return the sum

def add_two_numbers(num1, num2):
    sum = num1 + num2
    return sum





In [129]:
print(voc_out.stoi['('])

5


In [144]:
from torch.utils.data import Dataset
class Train_Dataset(Dataset):
    '''
    Initiating Variables
    df: the training dataframe
    source_column : the name of source text column in the dataframe
    target_columns : the name of target text column in the dataframe
    transform : If we want to add any augmentation
    freq_threshold : the minimum times a word must occur in corpus to be treated in vocab
    source_vocab_max_size : max source vocab size
    target_vocab_max_size : max target vocab size
    '''
    
    def __init__(self, df, source_column, target_column, transform=None, freq_threshold = 0,
                source_vocab_max_size = 10000, target_vocab_max_size = 10000):
    
        self.df = df
        self.transform = transform
        
        #get source and target texts
        self.source_texts = self.df[source_column]
        self.target_texts = self.df[target_column]
        
        
        ##VOCAB class has been created above
        #Initialize source vocab object and build vocabulary
        self.source_vocab = Vocabulary_Input(freq_threshold, source_vocab_max_size)
        self.source_vocab.build_vocabulary(self.source_texts.tolist())
        #Initialize target vocab object and build vocabulary
        self.target_vocab = Vocabulary_Output(freq_threshold, target_vocab_max_size)
        self.target_vocab.build_vocabulary(self.target_texts.tolist())
        
    def __len__(self):
        return len(self.df)
    
    '''
    __getitem__ runs on 1 example at a time. Here, we get an example at index and return its numericalize source and
    target values using the vocabulary objects we created in __init__
    '''
    def __getitem__(self, index):
        source_text = self.source_texts[index]
        target_text = self.target_texts[index]
        
        if self.transform is not None:
            source_text = self.transform(source_text)
            
        #numericalize texts ['<SOS>','cat', 'in', 'a', 'bag','<EOS>'] -> [1,12,2,9,24,2]
        numerialized_source = [self.source_vocab.stoi["<SOS>"]]
        numerialized_source += self.source_vocab.numericalize(source_text)
        numerialized_source.append(self.source_vocab.stoi["<EOS>"])
    
        numerialized_target = [self.target_vocab.stoi["<SOS>"]]
        numerialized_target += self.target_vocab.numericalize(target_text)
        numerialized_target.append(self.target_vocab.stoi["<EOS>"])
        
        #convert the list to tensor and return
        return torch.tensor(numerialized_source), torch.tensor(numerialized_target) 

class Validation_Dataset:
    def __init__(self, train_dataset, df, source_column, target_column, transform = None):
        self.df = df
        self.transform = transform
        
        #train dataset will be used as lookup for vocab
        self.train_dataset = train_dataset
        
        #get source and target texts
        self.source_texts = self.df[source_column]
        self.target_texts = self.df[target_column]
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self,index):
        source_text = self.source_texts[index]
        #print(source_text)
        target_text = self.target_texts[index]
        #print(target_text)
        if self.transform is not None:
            source_text = self.transform(source_text)
            
        #numericalize texts ['<SOS>','cat', 'in', 'a', 'bag','<EOS>'] -> [1,12,2,9,24,2]
        numerialized_source = [self.train_dataset.source_vocab.stoi["<SOS>"]]
        numerialized_source += self.train_dataset.source_vocab.numericalize(source_text)
        numerialized_source.append(self.train_dataset.source_vocab.stoi["<EOS>"])
     
        numerialized_target = [self.train_dataset.target_vocab.stoi["<SOS>"]]
        numerialized_target += self.train_dataset.target_vocab.numericalize(target_text)
        numerialized_target.append(self.train_dataset.target_vocab.stoi["<EOS>"])
        #print(numerialized_source)
        return torch.tensor(numerialized_source), torch.tensor(numerialized_target) 

In [158]:
train_dataset = Train_Dataset(train_df, 'question', 'solution')
idx = 10
print(train_df.loc[idx])
print(train_dataset[idx])

print(len(train_dataset[idx][0]), len(train_dataset[idx][1]))

print([train_dataset.source_vocab.itos[i.item()] for i in train_dataset[idx][0]])
print([train_dataset.target_vocab.itos[i.item()] for i in train_dataset[idx][1]])

0
question     Write a python function that returns the sum ...
solution    def sum_natural(num):\n    if num < 0:\n      ...
Name: 10, dtype: object
(tensor([  1,   5,   7,   4,   8,  11,  19,  37,   9,  51,  10,  41, 114,  24,
          2]), tensor([   1,   11,   19, 1400,    5,   37,    6,    9,    4,   17,   23,   37,
          64,   22,    9,    4,   94,   14,    5, 1401,    6,    4,    7,   35,
           9,    4,   94,   71,   10,   22,    4,   68,    5,   37,   58,   22,
           6,    9,    4,  182,   71,   59,   37,    4,   37,  224,   15,    4,
           7,   20,   37,    4,    4,    7,    7,    7,    2]))
15 57
['<SOS>', '', 'write', 'a', 'python', 'function', 'that', 'returns', 'the', 'sum', 'of', 'n', 'natural', 'numbers', '<EOS>']
['<SOS>', 'utf-8', 'def', 'sum_natural', '(', 'num', ')', ':', '\n', '    ', 'if', 'num', '<', '0', ':', '\n', '       ', 'print', '(', '"Please enter a positive number!"', ')', '\n', '', 'else', ':', '\n', '       ', 'sum', '=', '0', '\n', 

In [196]:
from torch.nn.utils.rnn import pad_sequence

# Desired max length
max_len = 50

# 100 seqs of variable length (< max_len)
seq_lens = torch.randint(low=10,high=44,size=(100,))
seqs = [torch.rand(n) for n in seq_lens]

print(seqs[0].shape)

# pad first seq to desired length
seqs[0] = nn.ConstantPad1d((0, max_len - seqs[0].shape[0]), 0)(seqs[0])

# pad all seqs to desired length
seqs = pad_sequence(seqs)
print(seqs)
print(seqs.shape)

torch.Size([37])
tensor([[0.0638, 0.2659, 0.1415,  ..., 0.7142, 0.5426, 0.2871],
        [0.2370, 0.4653, 0.9684,  ..., 0.9255, 0.8317, 0.3114],
        [0.7673, 0.6815, 0.8197,  ..., 0.1055, 0.5047, 0.3394],
        ...,
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]])
torch.Size([50, 100])


In [206]:
class MyCollate:
    def __init__(self, pad_idx):
        self.pad_idx = pad_idx
        
    #__call__: a default method
    ##   First the obj is created using MyCollate(pad_idx) in data loader
    ##   Then if obj(batch) is called -> __call__ runs by default
    def __call__(self, batch):
        #get all source indexed sentences of the batch


        source = [item[0] for item in batch] 
        #pad them using pad_sequence method from pytorch. 
        source = torch.nn.utils.rnn.pad_sequence(source, batch_first=False, padding_value = self.pad_idx) 
        
        #get all target indexed sentences of the batch
        target = [item[1] for item in batch] 
        #pad them using pad_sequence method from pytorch. 
        target = torch.nn.utils.rnn.pad_sequence(target, batch_first=False, padding_value = self.pad_idx)
        return source, target

In [207]:
# If we run a next(iter(data_loader)) we get an output of batch_size * (num_workers+1)
def get_train_loader(dataset, batch_size, num_workers=0, shuffle=True, pin_memory=True): #increase num_workers according to CPU
    #get pad_idx for collate fn
    pad_idx = dataset.source_vocab.stoi['<PAD>']
    #define loader
    loader = DataLoader(dataset, batch_size = batch_size, num_workers = num_workers,
                        shuffle=shuffle,
                       pin_memory=pin_memory, collate_fn = MyCollate(pad_idx=pad_idx)) #MyCollate class runs __call__ method by default
    return loader

def get_valid_loader(dataset, train_dataset, batch_size, num_workers=0, shuffle=True, pin_memory=True):
    pad_idx = train_dataset.source_vocab.stoi['<PAD>']
    loader = DataLoader(dataset, batch_size = batch_size, num_workers = num_workers,
                        shuffle=shuffle,
                       pin_memory=pin_memory, collate_fn = MyCollate(pad_idx=pad_idx))
    return loader

In [208]:
train_dataset[7]

(tensor([  1,   5,   7,   4,  12,   6,  16,   9, 146, 272,  10,   4,  18,  17,
           2]),
 tensor([   1,   11,    4,   37,   10,  110,    4,   27,   18,   24,   39,    5,
           15,    8,  183,    6,    9,    4,   83,   14,    5, 1390,    6,    4,
            4,    4,    7,    7,    2]))

In [263]:
train_loader = get_train_loader(train_dataset, 1)

source = next(iter(train_loader))[0]
target = next(iter(train_loader))[1]

print('source: \n', source)
print('target: \n', target)

print('source shape: ',source.shape)
print('target shape: ', target.shape)

source: 
 tensor([[  1],
        [  5],
        [  7],
        [  4],
        [  8],
        [ 12],
        [  6],
        [134],
        [ 34],
        [  9],
        [107],
        [ 15],
        [  4],
        [ 52],
        [  2]])
target: 
 tensor([[   1],
        [  11],
        [   4],
        [  19],
        [1403],
        [   5],
        [ 129],
        [   8],
        [ 166],
        [   6],
        [   9],
        [   4],
        [  17],
        [  20],
        [  52],
        [   5],
        [ 141],
        [   5],
        [ 129],
        [   8],
        [ 166],
        [   6],
        [   6],
        [   4],
        [   4],
        [   4],
        [   7],
        [   7],
        [   2]])
source shape:  torch.Size([15, 1])
target shape:  torch.Size([29, 1])


In [264]:
class Transformer(nn.Module):
    def __init__(
        self,
        embedding_size,
        src_vocab_size,
        trg_vocab_size,
        src_pad_idx,
        num_heads,
        num_encoder_layers,
        num_decoder_layers,
        forward_expansion,
        dropout,
        max_len,
        device,
    ):
        super(Transformer, self).__init__()
        self.src_word_embedding = nn.Embedding(src_vocab_size, embedding_size)
        self.src_position_embedding = nn.Embedding(max_len, embedding_size)
        self.trg_word_embedding = nn.Embedding(trg_vocab_size, embedding_size)
        self.trg_position_embedding = nn.Embedding(max_len, embedding_size)

        self.device = device
        self.transformer = nn.Transformer(
            embedding_size,
            num_heads,
            num_encoder_layers,
            num_decoder_layers,
            forward_expansion,
            dropout,
            batch_first=False
        )
        self.fc_out = nn.Linear(embedding_size, trg_vocab_size)
        self.dropout = nn.Dropout(dropout)
        self.src_pad_idx = src_pad_idx

    def make_src_mask(self, src):
        src_mask = src.transpose(0, 1) == self.src_pad_idx

        # (N, src_len)
        return src_mask.to(self.device)

    def forward(self, src, trg):
        src_seq_length, N = src.shape
        trg_seq_length, N = trg.shape

        src_positions = (
            torch.arange(0, src_seq_length)
            .unsqueeze(1)
            .expand(src_seq_length, N)
            .to(self.device)
        )

        trg_positions = (
            torch.arange(0, trg_seq_length)
            .unsqueeze(1)
            .expand(trg_seq_length, N)
            .to(self.device)
        )

        embed_src = self.dropout(
            (self.src_word_embedding(src) + self.src_position_embedding(src_positions))
        )
        embed_trg = self.dropout(
            (self.trg_word_embedding(trg) + self.trg_position_embedding(trg_positions))
        )

        src_padding_mask = self.make_src_mask(src)
        trg_mask = self.transformer.generate_square_subsequent_mask(trg_seq_length).to(
            self.device
        )
        print("here")
        out = self.transformer(
            embed_src,
            embed_trg,
            src_key_padding_mask=src_padding_mask,
            tgt_mask=trg_mask,
        )
        out = self.fc_out(out)
        return out

In [265]:
# We're ready to define everything we need for training our Seq2Seq model

load_model = False
save_model = False

# Training hyperparameters
num_epochs = 10000
learning_rate = 3e-4
batch_size = 1

# Model hyperparameters
src_vocab_size = len(train_dataset.source_vocab)
trg_vocab_size = len(train_dataset.target_vocab)
embedding_size = 512
num_heads = 8
num_encoder_layers = 3
num_decoder_layers = 3
dropout = 0.10
max_len = 100
forward_expansion = 4
src_pad_idx = train_dataset.source_vocab.stoi["<PAD>"]

In [266]:
model = Transformer(
    embedding_size,
    src_vocab_size,
    trg_vocab_size,
    src_pad_idx,
    num_heads,
    num_encoder_layers,
    num_decoder_layers,
    forward_expansion,
    dropout,
    max_len,
    device,
).to(device)

In [267]:
writer = SummaryWriter("runs/loss_plot")
step = 0

In [268]:
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, factor=0.1, patience=10, verbose=True
)
pad_idx = train_dataset.source_vocab.stoi["<PAD>"]
criterion = nn.CrossEntropyLoss(ignore_index=pad_idx)

In [269]:
""" if load_model:
    load_checkpoint(torch.load("my_checkpoint.pth.tar"), model, optimizer) """

for epoch in range(num_epochs):
    print(f"[Epoch {epoch} / {num_epochs}]")

    if save_model:
        checkpoint = {
            "state_dict": model.state_dict(),
            "optimizer": optimizer.state_dict(),
        }
        #save_checkpoint(checkpoint)

    model.eval()

    model.train()
    losses = []

    for batch_idx, batch in enumerate(train_loader):
        print("batch idx ",batch_idx)
        # Get input and targets and get to cuda
        #print(batch)
        inp_data,target = batch
        #inp_data = inp_data.to(device)
        #target = target.to(device)

        
        print(inp_data.shape,target.shape)
        print(inp_data.shape[0])

        #print(torch.full((512-inp_data.shape[0],1),3 ))

        inp_data = torch.cat( (inp_data, torch.full((512-inp_data.shape[0],1),3 )),0)
        target = torch.cat( (target, torch.full((512-target.shape[0],1),3)),0 )

        print(inp_data.shape,target.shape)
        target[:-1, :]
        output = model(inp_data,target )
        print("output:",output.shape)

        # Output is of shape (trg_len, batch_size, output_dim) but Cross Entropy Loss
        # doesn't take input in that form. For example if we have MNIST we want to have
        # output to be: (N, 10) and targets just (N). Here we can view it in a similar
        # way that we have output_words * batch_size that we want to send in into
        # our cost function, so we need to do some reshapin.
        # Let's also remove the start token while we're at it
        output = output.reshape(-1, output.shape[2])
        target = target[1:].reshape(-1)

        optimizer.zero_grad()

        loss = criterion(output, target)
        losses.append(loss.item())

        # Back prop
        loss.backward()
        # Clip to avoid exploding gradient issues, makes sure grads are
        # within a healthy range
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1)

        # Gradient descent step
        optimizer.step()

        # plot to tensorboard
        writer.add_scalar("Training loss", loss, global_step=step)
        step += 1

    mean_loss = sum(losses) / len(losses)
    scheduler.step(mean_loss)

[Epoch 0 / 10000]
batch idx  0
torch.Size([18, 1]) torch.Size([31, 1])
18
torch.Size([512, 1]) torch.Size([512, 1])


IndexError: index out of range in self